In [1]:
import os, random, uuid, datetime
from decimal import Decimal, ROUND_HALF_UP
from pymongo import MongoClient
from bson.decimal128 import Decimal128

PH_TZ = datetime.timezone(datetime.timedelta(hours=8))

mongo_uri = os.getenv("MONGODB_URI")
db_name = os.getenv("MONGODB_DATABASE")
client = MongoClient(mongo_uri, tz_aware=True)
db = client[db_name]
col = db["pump_inventories"]

loboc_map = [
  {"dispenser_id": 3, "dispenser_name": "dispenser-three", "pump_id": "5", "product": "regular", "location": "loboc"},
  {"dispenser_id": 3, "dispenser_name": "dispenser-three", "pump_id": "6", "product": "diesel",  "location": "loboc"},
  {"dispenser_id": 4, "dispenser_name": "dispenser-four",  "pump_id": "7", "product": "regular", "location": "loboc"},
  {"dispenser_id": 4, "dispenser_name": "dispenser-four",  "pump_id": "8", "product": "diesel",  "location": "loboc"},
]

prices = {"regular": Decimal("62.50"), "diesel": Decimal("58.00")}
unit = "liter"

def ph_start_to_utc(d):
    dt_ph = datetime.datetime(d.year, d.month, d.day, 0, 0, 0, tzinfo=PH_TZ)
    return dt_ph.astimezone(datetime.timezone.utc)

def as_dec128(x):
    return Decimal128(Decimal(x).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP))

date_ph = datetime.date(2025, 9, 28)

random.seed(20250928)
docs = []
for m in loboc_map:
    meter_start = Decimal(str(random.randint(20000, 50000)))
    doc = {
        "id": str(uuid.uuid4()),
        "created_at": datetime.datetime.now(datetime.timezone.utc),
        "location": m["location"],
        "date": ph_start_to_utc(date_ph),
        "dispenser_name": m["dispenser_name"],
        "pump_id": str(m["pump_id"]),
        "pump_name": m["product"],
        "product": m["product"],
        "unit": unit,
        "price": as_dec128(prices[m["product"]]),
        "beginning_inventory": as_dec128("5000"),
        "calibration": as_dec128("0"),
        "po": as_dec128("0"),
        "cash": as_dec128("0"),
        "ending_inventory": as_dec128("5000"),
        "starting_liter_meter": as_dec128(meter_start),
        "ending_liter_meter": as_dec128(meter_start),
    }
    docs.append(doc)

if docs:
    col.insert_many(docs)

len(docs)


4